# Working with MPAS Meshes

##  Mesh Definition

All components of MPAS lie on a C-grid staggered Voronoi Mesh as seen in the figure below:

<p align="center">
  <img src="../_static/examples/mpas/c-grid.png"
  width="400" / >
</p>

This type of mesh is represented through two different sub-meshes. The Primal Mesh represents the Voronoi Mesh and the Dual Mesh represents the Triangular Mesh.

*to-do: more in-depth description/overview of MPAS from the website and specification document.*




## Imports

In [ ]:
import xarray as xr
import uxarray as ux

## Grid and Data Files

As mentioned in earlier notebooks, the grid definition and data variables are typically stored in separate files. However, in this example, our dataset will contain both within the same file, which is often the case when working with smaller datasets.



In [ ]:
mpas_root_filepath = "../../test/meshfiles/mpas/"
mpas_dataset_filepath = mpas_root_filepath + "QU/mesh.QU.1920km.151026.nc"

In [ ]:
mpas_xr_ds = xr.open_dataset(mpas_dataset_filepath)
mpas_xr_ds

This dataset contains the full set of connectivity variables needed to describe an MPAS mesh, as described in the MPAS Specification Document [2]. It also contains data variables such as {VARIABLE} and , which lie on the Primal Mesh. Below is a list of connectivity variables and their geometric meaning, which will be further described when comparing to the UGRID encoding:


### Primal Mesh
* **lonVertex, latVertex**: Corner Vertices of Primal Mesh cells
* **lonCell, latCell**: Center Vertices of Primal Mesh cells
* **verticesOnCell**: Vertex indices that surround each Primal Mesh cell
* **verticesOnEdge**: Vertex indices that saddle a given edge
* **nEdgesOnCell**: Maximum number of edges that can surround a cell

### Dual Mesh
* **lonCell, latCell**: Corner Vertices of Dual Mesh cells
* **lonVertex, latVertex**: Center Vertices of Dual Mesh cells
* **cellsOnVertex**: Vertex indices that surround each Dual Mesh cell
* **cellsOnEdge**: Vertex indices that saddle a given edge


## Constructing a Grid Object

The `xarray.Dataset` that we opened above stores the coordinates and connectivity variables for both the Primal and Dual meshes together in a single dataset. With `uxarray`, we can create separate `Grid` objects for either the Primal or Dual meshes. The `uxarray.Grid` class parses our input dataset, detects that it is represented in the MPAS specifications, and encodes either the Primal or Dual mesh in the UGRID conventions.

In [ ]:
primal_mesh = ux.Grid(mpas_xr_ds, use_dual=False)
dual_mesh = ux.Grid(mpas_xr_ds, use_dual=True)

In [ ]:
primal_mesh.ds

In [ ]:
dual_mesh.ds

## Relationship between MPAS and UGRID
*to-do: describe MPAS to UGRID encoding, differences in fill values, standardized datatypes, and how geometric connectivity variables relate between these two conventions*

*create a table to showcase the mapping between MPAS and UGRID*

## Functionality

### Face Area

In [ ]:
primal_mesh_total_area = primal_mesh.calculate_total_face_area()
primal_mesh_total_area

In [ ]:
primal_mesh_face_areas = primal_mesh.face_areas
primal_mesh_face_areas.sum()

In [ ]:
dual_mesh_total_area = dual_mesh.calculate_total_face_area()
dual_mesh_total_area

In [ ]:
dual_mesh_face_areas = dual_mesh.face_areas
primal_mesh_face_areas.sum()

### Integration

In [ ]:
# to-do, need to fix function before adding

## Visualization

*very messy, waiting on other PR's to get merged before doing primal mesh*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

### Primal Mesh

In [ ]:
x = primal_mesh.Mesh2_node_x.values
y = primal_mesh.Mesh2_node_y.values
face_nodes = primal_mesh.Mesh2_face_nodes.values
# face_dimension = primal_mesh.Mesh2_face_dimension.values

# polygons = collections.PolyCollection(vertices)

### Dual Mesh

In [ ]:
n_face = dual_mesh.nMesh2_face
x = dual_mesh.Mesh2_node_x.values
y = dual_mesh.Mesh2_node_y.values
face_nodes = dual_mesh.Mesh2_face_nodes.values

triangle_x = np.zeros((n_face, 3 + 1))
triangle_y = np.zeros((n_face, 3 + 1))
triangle_x[:, 0:3] = x[face_nodes]
triangle_y[:, 0:3] = y[face_nodes]
triangle_x[:, -1] = triangle_x[:, 0]
triangle_y[:, -1] = triangle_y[:, 0]

remove_idx = np.any(np.abs(triangle_x - 180) > 170 , axis=1)

verts = np.stack([triangle_x[~remove_idx], triangle_y[~remove_idx]])
verts = np.swapaxes(verts, 0, 1)
verts = np.swapaxes(verts, 1, 2)


polygons = PolyCollection(verts, edgecolors='Blue')
colors = 100 * np.random.rand(len(verts))

fig, ax = plt.subplots()

ax.add_collection(polygons, autolim=False)
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())
ax.set_title("Dual Mesh")

## References

[1] https://mpas-dev.github.io/

[2] https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf

[3] http://ugrid-conventions.github.io/ugrid-conventions/